In [264]:
import pandas as pd 
import pandas_ta as ta 
import yfinance as yf 
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio

In [265]:
def cleaning(df):
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    df.columns = [name[0].lower() for name in df.columns]  # Convert column names to lowercase
    df.index = df.index.rename('date')  # Rename index to 'date'
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d %H:%M:%S')  # Format index as datetime string
    df = df.reset_index(drop=False)  # Reset index and assign back to df
    return df

In [266]:
def chart(df, fib_df=None):
    fig = go.Figure(data=[go.Candlestick(
        x=df['date'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    )])

    fig.update_layout(
        xaxis_rangeslider_visible=False
    )
    print('successfull candlestick plot')

    if fib_df is not None and not fib_df.empty:
        print('Fibonacci dataframe was given and its not empty')

        fib_df = fib_df.iloc[:-1] if fib_df.iloc[-1].isna().any() else fib_df
        
        # Add yellow boxes (assumed example ranges)
        for _, row in fib_df.iterrows():
            fig.add_shape(
                type="rect",
                x0=row['date'],
                x1=row['next_day'],
                y0=row['fib 61.8'],
                y1=row['stop_loss'],
                fillcolor="yellow" if row['trend'] == -1 else 'red',
                opacity=0.3,
                line_width=0
            )
        print('fibonacci levels were ploted')

    # Show the chart in a new browser tab
    pio.show(fig, renderer='browser')


In [267]:
gold_daily = cleaning(yf.download('GC=F', period='60d', interval='1d'))

[*********************100%***********************]  1 of 1 completed


In [312]:
gold_15 = cleaning(yf.download('GC=F',period='8d', interval='15m'))

[*********************100%***********************]  1 of 1 completed


In [269]:
gold_1h = cleaning(yf.download('GC=F',period='60d', interval='1h'))

[*********************100%***********************]  1 of 1 completed


In [270]:
gold_daily.head()

,date,close,high,low,open,volume
0,2024-11-18 00:00:00,2610.600098,2610.699951,2589.399902,2591.699951,136
1,2024-11-19 00:00:00,2627.100098,2627.699951,2619.300049,2621.899902,87
2,2024-11-20 00:00:00,2648.199951,2648.199951,2638.500000,2638.500000,27
3,2024-11-21 00:00:00,2672.100098,2672.100098,2655.699951,2659.300049,33
4,2024-11-22 00:00:00,2709.899902,2710.500000,2685.600098,2687.500000,41


In [320]:
gold_15.head(3)

,date,close,high,low,open,volume
0,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
2,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582


In [272]:
def get_daily_fibonacci(df, levels_only=False, stop_loss=20):
    
    # Add a trend column: 1 for uptrend, -1 for downtrend
    df['trend'] = df.apply(lambda row: 1 if row['close'] > row['open'] else -1, axis=1)

    # Create new columns and calculate Fibonacci levels for each row
    df['fib 23.6'] = df['high'] - ((df['high'] - df['low']) * 0.236)
    df['fib 38.2'] = df['high'] - ((df['high'] - df['low']) * 0.382)
    df['fib 61.8'] = df['high'] - ((df['high'] - df['low']) * 0.618)
    df['fib 100.0'] = df['low']  # 100% Fibonacci level corresponds to the low price

    # Calculate next day's index
    # Shift the 'date' column to create 'next_day'
    df['next_day'] = df['date'].shift(-1)
    print(df['next_day'].dtype)
    # Replace NaT with the next day's date
    df['next_day'] = df['next_day'].fillna(pd.to_datetime(df['date'].iloc[-1]) + pd.Timedelta(days=1))


    # Calculate stop_loss based on the trend and Fibonacci levels
    df['stop_loss'] = df.apply(
        lambda row: (row['fib 61.8'] - stop_loss) if row['trend'] == 1 else (row['fib 61.8'] + stop_loss), axis=1
    )

    # Return only Fibonacci levels and trend if levels_only=True
    if levels_only:
        return df[["fib 23.6", 'fib 38.2', 'fib 61.8', 'fib 100.0', 'trend', 'next_day', 'stop_loss']]
    else:
        return df


In [273]:
gold_daily_fib = get_daily_fibonacci(gold_daily)

object


In [274]:
gold_daily_fib.tail(3)

,date,close,high,low,open,volume,trend,fib 23.6,fib 38.2,fib 61.8,fib 100.0,next_day,stop_loss
45,2025-01-23 00:00:00,2763.100098,2765.000000,2745.399902,2760.000000,2201,1,2760.374377,2757.512763,2752.887140,2745.399902,2025-01-24 00:00:00,2732.887140
46,2025-01-24 00:00:00,2777.300049,2792.000000,2759.800049,2759.800049,2201,1,2784.400812,2779.699619,2772.100430,2759.800049,2025-01-28 00:00:00,2752.100430
47,2025-01-28 00:00:00,2766.800049,2768.899902,2737.399902,2742.399902,142303,1,2761.465902,2756.866902,2749.432902,2737.399902,2025-01-29 00:00:00,2729.432902


In [321]:
chart(gold_15, fib_df=gold_daily_fib)

successfull candlestick plot
Fibonacci dataframe was given and its not empty
fibonacci levels were ploted


In [276]:
chart(gold_1h, fib_df=gold_daily_fib)

successfull candlestick plot
Fibonacci dataframe was given and its not empty
fibonacci levels were ploted


# BackTesting.py

In [277]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

In [278]:
class Fib_daily(Strategy):

    def init(self):
        # Access data series
        self.fib = self.data['fib 61.8']  # Ensure 'fib 61.8' is a valid column in your data
        # print('fibonacci', self.fib)

        self.close = self.data.Close
        # print('close data',self.data.Close)

        self.previous_close = [self.data.Close[i-1] for i in range(1, len(self.data.Close))]
        # print('previous', self.previous_close)

        self.stop_loss = self.data.stop_loss

    def next(self):
        # Check if the previous close was above the Fibonacci level and the current close is below it 
        if self.previous_close[0] >= self.fib and self.close <= self.fib:
            self.buy()  # Enter a long position
            self.position.stop_loss = self.stop_loss
            self.position.take_profit = self.fib + 30
            self.previous_close.pop(0)

        # Check if the previous close was below the Fibonacci level and the current close is above it
        elif self.previous_close[0] < self.fib and self.close >= self.fib:
            self.sell()  # Exit the position or go short
            self.position.stop_loss = self.stop_loss
            self.position.take_profit = self.fib + 30
            self.previous_close.pop(0)



In [279]:
gold_daily.head(3)

,date,close,high,low,open,volume,trend,fib 23.6,fib 38.2,fib 61.8,fib 100.0,next_day,stop_loss
0,2024-11-18 00:00:00,2610.600098,2610.699951,2589.399902,2591.699951,136,1,2605.673140,2602.563333,2597.536521,2589.399902,2024-11-19 00:00:00,2577.536521
1,2024-11-19 00:00:00,2627.100098,2627.699951,2619.300049,2621.899902,87,1,2625.717574,2624.491188,2622.508812,2619.300049,2024-11-20 00:00:00,2602.508812
2,2024-11-20 00:00:00,2648.199951,2648.199951,2638.500000,2638.500000,27,1,2645.910763,2644.494570,2642.205381,2638.500000,2024-11-21 00:00:00,2622.205381


## Preparing data for backtesting

In [280]:
def prepare_for_backtesting(df):
    df = df.copy()
    df = df.rename(columns=dict(open='Open', high='High', low='Low', close='Close'))
    df = df.set_index('date')
    print(df.index.dtype)
    df.index = pd.DatetimeIndex(df.index)
    print(df.index.dtype)

    return df


In [281]:
data = prepare_for_backtesting(gold_daily)

object
datetime64[ns]


In [282]:
# Create a backtest instance
bt = Backtest(data, Fib_daily, cash=10000, commission=.002)

# Run the backtest
stats = bt.run()
print(stats)

Start                     2024-11-18 00:00:00
End                       2025-01-28 00:00:00
Duration                     71 days 00:00:00
Exposure Time [%]                    95.83333
Equity Final [$]                   9672.46929
Equity Peak [$]                   10123.06915
Return [%]                           -3.27531
Buy & Hold Return [%]                  5.9833
Return (Ann.) [%]                   -16.04025
Volatility (Ann.) [%]                10.90693
CAGR [%]                            -11.14788
Sharpe Ratio                         -1.47065
Sortino Ratio                        -1.92042
Calmar Ratio                         -2.92412
Max. Drawdown [%]                    -5.48549
Avg. Drawdown [%]                    -3.75953
Max. Drawdown Duration       40 days 00:00:00
Avg. Drawdown Duration       24 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                       -4.14613
Worst Trade [%]                   

In [283]:
bt.plot()

GridPlot(id='p1571', ...)

In [284]:
gold_15.set_index('date').index.dtype

dtype('O')

In [311]:
gold_15

,date,close,high,low,open,volume
1970-01-01 00:00:00.000000000,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1970-01-01 00:00:00.000000001,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
1970-01-01 00:00:00.000000002,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582
1970-01-01 00:00:00.000000003,2025-01-20 00:45:00,2749.100098,2749.300049,2746.800049,2748.899902,1296
1970-01-01 00:00:00.000000004,2025-01-20 01:00:00,2749.699951,2750.500000,2748.300049,2749.000000,623
...,...,...,...,...,...,...
1970-01-01 00:00:00.000000589,2025-01-28 11:45:00,2766.300049,2767.399902,2763.100098,2763.100098,4447
1970-01-01 00:00:00.000000590,2025-01-28 12:00:00,2768.000000,2768.600098,2764.800049,2766.300049,2663
1970-01-01 00:00:00.000000591,2025-01-28 12:15:00,2767.199951,2768.899902,2766.600098,2767.899902,1940
1970-01-01 00:00:00.000000592,2025-01-28 12:30:00,2765.899902,2767.300049,2765.000000,2767.300049,2587


In [318]:
def d1_fibonacci(df):
    df.set_index('date')
    # df.index = pd.DatetimeIndex(df.index)
    
    # df_daily = df.resample('D').agg({
    #     "open": 'first',
    #     'high': 'max',
    #     'low':  'min',
    #     'close': 'last'
    # })

    # # Create new columns and calculate Fibonacci levels for each row
    # df_daily['fib 61.8'] = df_daily['high'] - ((df_daily['high'] - df_daily['low']) * 0.618)

    # df = df_daily

    return df

In [319]:
d1_fibonacci(gold_15)

,date,close,high,low,open,volume
0,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
2,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582
3,2025-01-20 00:45:00,2749.100098,2749.300049,2746.800049,2748.899902,1296
4,2025-01-20 01:00:00,2749.699951,2750.500000,2748.300049,2749.000000,623
...,...,...,...,...,...,...
592,2025-01-28 12:30:00,2765.899902,2767.300049,2765.000000,2767.300049,2587
593,2025-01-28 12:45:00,2767.100098,2768.000000,2764.800049,2765.800049,3744
594,2025-01-28 13:00:00,2768.300049,2768.800049,2766.800049,2767.100098,2445
595,2025-01-28 13:15:00,2767.399902,2768.600098,2767.100098,2768.300049,3076
